In [1]:
import json
import pandas as pd
from tqdm import tqdm

import data_process

In [74]:
!du -hs data/train/0_1_1000_train.parquet.gzip

/bin/bash: /home/hilbert/anaconda3/lib/libtinfo.so.6: no version information available (required by /bin/bash)
420K	data/train/0_1_1000_train.parquet.gzip


In [12]:
import os
clms_to_leave = ["site-id", "session-duration", "pages-count", "target"]

df = pd.DataFrame()
for f in os.listdir("data/train/"):
    dfs = pd.read_parquet(f"data/train/{f}")
    dfs = dfs[clms_to_leave]
    dfs = dfs.drop_duplicates().reset_index(drop=True)

    df = pd.concat([df,dfs]).reset_index(drop=True)
df.shape

(1242277, 4)

In [14]:
dfval = pd.DataFrame()
for f in os.listdir("data/val/"):
    dfs = pd.read_parquet(f"data/val/{f}")
    dfs = dfs[clms_to_leave]
    dfs = dfs.drop_duplicates().reset_index(drop=True)

    dfval = pd.concat([dfval,dfs]).reset_index(drop=True)
    
dfval.shape

(263091, 4)

In [35]:
from catboost import CatBoostClassifier
from sklearn.model_selection import train_test_split

df["site-id"] = df["site-id"].astype(int).astype(str)

xtr, xvl, ytr, yvl = train_test_split(
    df.drop("target", axis=1),
    df.target, 
    test_size=0.2, random_state=42
)

ytr = (ytr == "male").astype(int)
yvl = (yvl == "male").astype(int)

model = CatBoostClassifier(iterations=2000, depth=10, learning_rate=0.05, loss_function='Logloss', verbose=1)
model.fit(xtr, ytr, eval_set=(xvl, yvl), cat_features=["site-id"], plot=True)

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

0:	learn: 0.6865876	test: 0.6865480	best: 0.6865480 (0)	total: 160ms	remaining: 5m 20s
1:	learn: 0.6807621	test: 0.6806808	best: 0.6806808 (1)	total: 314ms	remaining: 5m 14s
2:	learn: 0.6755113	test: 0.6753748	best: 0.6753748 (2)	total: 488ms	remaining: 5m 24s
3:	learn: 0.6708926	test: 0.6706657	best: 0.6706657 (3)	total: 634ms	remaining: 5m 16s
4:	learn: 0.6667557	test: 0.6664892	best: 0.6664892 (4)	total: 843ms	remaining: 5m 36s
5:	learn: 0.6630239	test: 0.6627054	best: 0.6627054 (5)	total: 1s	remaining: 5m 33s
6:	learn: 0.6597095	test: 0.6593618	best: 0.6593618 (6)	total: 1.16s	remaining: 5m 30s
7:	learn: 0.6567690	test: 0.6563884	best: 0.6563884 (7)	total: 1.3s	remaining: 5m 24s
8:	learn: 0.6541693	test: 0.6537585	best: 0.6537585 (8)	total: 1.52s	remaining: 5m 36s
9:	learn: 0.6518467	test: 0.6513976	best: 0.6513976 (9)	total: 1.65s	remaining: 5m 27s
10:	learn: 0.6496819	test: 0.6492190	best: 0.6492190 (10)	total: 1.8s	remaining: 5m 25s
11:	learn: 0.6477416	test: 0.6472535	best: 0.6

In [32]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix

dfval["site-id"] = dfval["site-id"].astype(int).astype(str)

xts = dfval.drop("target", axis=1)
yts = dfval.target
yts = (yts == "male").astype(int)

ypr = model.predict(xts)

# Evaluate the model
accuracy = accuracy_score(yts, ypr)
precision = precision_score(yts, ypr)
recall = recall_score(yts, ypr)
f1 = f1_score(yts, ypr)
roc_auc = roc_auc_score(yts, model.predict_proba(xts)[:, 1])
conf_matrix = confusion_matrix(yts, ypr)

# Store the metrics in a dictionary
metrics_dict = {
    'Accuracy': accuracy,
    'Precision': precision,
    'Recall': recall,
    'F1 Score': f1,
    'ROC AUC': roc_auc,
    'Confusion Matrix': conf_matrix.tolist()  # Convert confusion matrix to a list for better readability
}

metrics_dict

{'Accuracy': 0.638117609496334,
 'Precision': 0.5939432276429193,
 'Recall': 0.6313860637135412,
 'F1 Score': 0.6120925684485006,
 'ROC AUC': 0.6966750765834292,
 'Confusion Matrix': [[92767, 51354], [43854, 75116]]}